In [1]:
from __future__ import print_function
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
## Disable tf future deprecated messages
import logging
logging.getLogger('tensorflow').disabled = True
## Disable tf CUDA messages
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

%matplotlib inline

import sys
import random
import numpy as np
import pandas as pd

import utils
import correction
from models.dense import *
from models.gain import gain
from models.soft_impute import SoftImpute
from models.sinkhorn import OTimputer
from models.mida import mida

import scipy.stats

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer, SimpleImputer

import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
from missingpy import MissForest

torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True
dtype = torch.cuda.FloatTensor

## Repeat all experiments for 10 runs

In [2]:
n_runs = 10

## Load data

In [3]:
data_missing, missing_mask, y = utils.load_nhanes(balanced=True)
## Inject noise
data_missing = utils.inject_noise(data_missing.copy(), noise_rate=.20)
## Replace missing values locations by 0
data_missing = data_missing * missing_mask

Dataset shape: (2000, 95)
33.65% missing data
Class distribution: (array([1., 2.]), array([1000, 1000]))


## Run OUR METHOD

In [4]:
params = {
    'nb_batches': 10,
    'reg_noise_std': .03,
    'net_input': 'data_corrupted',
    'net_params': {
        'input_channels':1,
        'output_channels':1,
        'channels_skip':4,
        'down_layers':[(24, 7, 1),
                       (46, 9, 5),
                       (96, 11, 1),
                       (96, 13, 1)],
        'need_sigmoid':True,
        'need_bias':True,
        'pad':'zero',
        'downsample_mode':'stride',
        'upsample_mode':'nearest',
        'act_fun':'LeakyReLU',
        'need1x1_up':True
    },# or a list containing layers sizes
    'adam_lr': .0001,
    'adam_weight_decay': 0.,
}

ours_accs, ours_aucs = [], []
for i in range(n_runs):
    ours_correction = correction.run(data_missing, 501, params, y=y, missing_mask=missing_mask, seed=i)
    ## Since our method returns 2 imputations select the best one
    scores_raw = utils.get_scores(ours_correction['raw_out'], y)
    acc = scores_raw['test_balanced_accuracy']
    auc = scores_raw['test_roc_auc_ovo']
    acc = acc.mean()
    acc_std = acc.std()
    auc = auc.mean()
    auc_std = auc.std()
    if 'masked_out' in ours_correction:
        scores_masked = utils.get_scores(ours_correction['masked_out'], y)
        acc_masked = scores_masked['test_balanced_accuracy']
        auc_masked = scores_masked['test_roc_auc_ovo']
        if auc_masked.mean() > auc:
            acc = acc_masked.mean()
            acc_std = acc_masked.std()
            auc = auc_masked.mean()
            auc_std = auc_masked.std()
    print(f'OUR METHOD RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%')
    ours_accs.append(acc.mean())
    ours_aucs.append(auc.mean())
print(f'\nOUR METHOD GLOBAL - acc: {round(np.array(ours_accs).mean() * 100, 2)}% +- {round(np.array(ours_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(ours_aucs).mean() * 100, 2)}% +- {round(np.array(ours_aucs).std() * 100, 2)}%')

Ite 00000 - 4.76 sec - Loss 0.061286 - ACC 59.40% - ACC Mean 59.40% - AUC 63.53% - AUC Mean 63.53% - Deter 000
Ite 00050 - 0.94 sec - Loss 0.012290 - ACC 60.60% - ACC Mean 59.43% - AUC 64.02% - AUC Mean 63.42% - Deter 043
Ite 00057 - 0.93 sec - Loss 0.010934 - ACC 60.35% - ACC Mean 59.38% - AUC 63.57% - AUC Mean 63.34% - Deter 050
Early stop ite 57, rollback to correction of ite 7, whith acc of 61.9% and auc of 65.78%
OUR METHOD RUN 1/10 - acc: 61.9% +- 0.0% - auc: 65.78% +- 0.0%
Ite 00000 - 1.17 sec - Loss 0.135436 - ACC 60.25% - ACC Mean 60.25% - AUC 63.69% - AUC Mean 63.69% - Deter 000
Ite 00050 - 1.06 sec - Loss 0.051122 - ACC 61.65% - ACC Mean 59.88% - AUC 64.73% - AUC Mean 63.68% - Deter 003
Ite 00100 - 1.14 sec - Loss 0.032109 - ACC 59.30% - ACC Mean 60.09% - AUC 62.89% - AUC Mean 64.13% - Deter 001
Ite 00150 - 1.09 sec - Loss 0.023171 - ACC 59.45% - ACC Mean 60.29% - AUC 62.84% - AUC Mean 64.45% - Deter 003
Ite 00197 - 1.01 sec - Loss 0.017711 - ACC 61.20% - ACC Mean 60.32% - A

## Run Median, Mean and KNN

In [5]:
data_missing_nans = np.where(missing_mask, data_missing, np.nan)

median_imputer = SimpleImputer(missing_values=np.nan, strategy='median')
mean_imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
knn_imputer = KNNImputer(n_neighbors=10, weights='uniform')

median_completed = median_imputer.fit_transform(data_missing_nans)
mean_completed = mean_imputer.fit_transform(data_missing_nans)
knn_completed = knn_imputer.fit_transform(data_missing_nans)

## All runs would be the same since deterministic methods
scores = utils.get_scores(median_completed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'MEDIAN - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
median_accs = acc
median_aucs = auc

scores = utils.get_scores(mean_completed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'MEAN - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
mean_accs = acc
mean_aucs = auc

scores = utils.get_scores(knn_completed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'KNN - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
knn_accs = acc
knn_aucs = auc

MEDIAN - acc: 59.4% +- 0.0% - auc: 64.48% +- 0.0%
MEAN - acc: 60.0% +- 0.0% - auc: 64.02% +- 0.0%
KNN - acc: 58.3% +- 0.0% - auc: 60.78% +- 0.0%


## Run GAIN

In [6]:
gain_parameters = {'batch_size': 128,
                   'hint_rate': .9,
                   'alpha': 100,
                   'iterations': 5000}

gain_accs, gain_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputed = gain(data_missing_nans, gain_parameters)
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'\nGAIN RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%\n')
    gain_accs.append(acc.mean())
    gain_aucs.append(auc.mean())
print(f'GAIN GLOBAL - acc: {round(np.array(gain_accs).mean() * 100, 2)}% +- {round(np.array(gain_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(gain_aucs).mean() * 100, 2)}% +- {round(np.array(gain_aucs).std() * 100, 2)}%')

100%|██████████| 5000/5000 [00:44<00:00, 112.80it/s]



GAIN RUN 1/10 - acc: 61.95% +- 2.31% - auc: 65.16% +- 3.46%



100%|██████████| 5000/5000 [00:49<00:00, 100.59it/s]



GAIN RUN 2/10 - acc: 60.5% +- 3.19% - auc: 65.01% +- 3.02%



100%|██████████| 5000/5000 [00:52<00:00, 96.04it/s]


...


GAIN RUN 8/10 - acc: 59.3% +- 4.14% - auc: 63.28% +- 4.53%



100%|██████████| 5000/5000 [01:06<00:00, 75.16it/s] 



GAIN RUN 9/10 - acc: 57.95% +- 1.68% - auc: 61.8% +- 2.06%



100%|██████████| 5000/5000 [01:07<00:00, 74.14it/s]



GAIN RUN 10/10 - acc: 62.45% +- 2.58% - auc: 66.02% +- 3.34%

GAIN GLOBAL - acc: 60.38% +- 1.48% - auc: 64.32% +- 1.49%


## Run MICE

In [7]:
imputer = IterativeImputer()
imputed = imputer.fit_transform(data_missing_nans)
## All runs would be the same since deterministic method
scores = utils.get_scores(imputed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'MICE - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
mice_accs = acc
mice_aucs = auc

MICE - acc: 59.6% +- 0.0% - auc: 63.03% +- 0.0%


## Run MISSFOREST

In [8]:
miss_accs, miss_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputer = MissForest(n_jobs=-1, random_state=i)
    imputed = imputer.fit_transform(data_missing_nans)
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'\nMISSFOREST RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%\n')
    miss_accs.append(acc.mean())
    miss_aucs.append(auc.mean())
print(f'\nMISSFOREST GLOBAL - acc: {round(np.array(miss_accs).mean() * 100, 2)}% +- {round(np.array(miss_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(miss_aucs).mean() * 100, 2)}% +- {round(np.array(miss_aucs).std() * 100, 2)}%')

Iteration: 0
Iteration: 1

MISSFOREST RUN 1/10 - acc: 58.6% +- 1.83% - auc: 62.34% +- 1.35%

Iteration: 0
Iteration: 1

MISSFOREST RUN 2/10 - acc: 57.6% +- 2.75% - auc: 61.11% +- 2.15%

Iteration: 0
Iteration: 1
Iteration: 2

MISSFOREST RUN 3/10 - acc: 59.0% +- 2.43% - auc: 63.43% +- 1.6%

Iteration: 0
Iteration: 1

MISSFOREST RUN 4/10 - acc: 60.4% +- 2.0% - auc: 64.55% +- 2.05%

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6

MISSFOREST RUN 5/10 - acc: 59.6% +- 1.79% - auc: 62.88% +- 1.75%

Iteration: 0
Iteration: 1

MISSFOREST RUN 6/10 - acc: 60.2% +- 1.93% - auc: 63.56% +- 1.62%

Iteration: 0
Iteration: 1

MISSFOREST RUN 7/10 - acc: 60.2% +- 1.5% - auc: 63.89% +- 2.17%

Iteration: 0
Iteration: 1

MISSFOREST RUN 8/10 - acc: 60.6% +- 2.02% - auc: 64.86% +- 0.97%

Iteration: 0
Iteration: 1

MISSFOREST RUN 9/10 - acc: 59.7% +- 3.22% - auc: 63.08% +- 4.09%

Iteration: 0
Iteration: 1

MISSFOREST RUN 10/10 - acc: 59.3% +- 1.35% - auc: 63.09% +- 2

## Run SOFTIMPUTE

In [9]:
soft_accs, soft_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputer = SoftImpute(random_state=i)
    imputer.fit(data_missing_nans)
    imputed = imputer.predict(data_missing_nans)
    imputed = np.where(missing_mask, data_missing, imputed)
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'SOFTIMPUTE RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%')
    soft_accs.append(acc.mean())
    soft_aucs.append(auc.mean())
print(f'\nSOFTIMPUTE GLOBAL - acc: {round(np.array(soft_accs).mean() * 100, 2)}% +- {round(np.array(soft_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(soft_aucs).mean() * 100, 2)}% +- {round(np.array(soft_aucs).std() * 100, 2)}%')

SOFTIMPUTE RUN 1/10 - acc: 59.65% +- 2.43% - auc: 63.68% +- 2.34%
SOFTIMPUTE RUN 2/10 - acc: 60.7% +- 2.06% - auc: 64.52% +- 2.5%
SOFTIMPUTE RUN 3/10 - acc: 61.95% +- 2.96% - auc: 66.03% +- 3.3%
SOFTIMPUTE RUN 4/10 - acc: 60.7% +- 3.81% - auc: 63.34% +- 4.79%
SOFTIMPUTE RUN 5/10 - acc: 60.75% +- 2.71% - auc: 64.98% +- 1.67%
SOFTIMPUTE RUN 6/10 - acc: 60.75% +- 3.32% - auc: 63.86% +- 3.05%
SOFTIMPUTE RUN 7/10 - acc: 57.85% +- 1.75% - auc: 62.19% +- 2.13%
SOFTIMPUTE RUN 8/10 - acc: 59.95% +- 1.92% - auc: 62.79% +- 2.73%
SOFTIMPUTE RUN 9/10 - acc: 60.55% +- 1.39% - auc: 64.41% +- 2.5%
SOFTIMPUTE RUN 10/10 - acc: 61.35% +- 3.18% - auc: 64.87% +- 2.75%

SOFTIMPUTE GLOBAL - acc: 60.42% +- 1.05% - auc: 64.07% +- 1.07%


## Run SINKHORN

In [10]:
sink_accs, sink_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputer = OTimputer(niter=500)
    imputed = imputer.fit_transform(data_missing_nans).cpu().detach().numpy()
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'\nSINKHORN RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%\n')
    sink_accs.append(acc.mean())
    sink_aucs.append(auc.mean())
print(f'\nSINKHORN GLOBAL - acc: {round(np.array(sink_accs).mean() * 100, 2)}% +- {round(np.array(sink_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(sink_aucs).mean() * 100, 2)}% +- {round(np.array(sink_aucs).std() * 100, 2)}%')

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 2.7121

SINKHORN RUN 1/10 - acc: 58.95% +- 2.34% - auc: 63.28% +- 1.59%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 2.7521

SINKHORN RUN 2/10 - acc: 58.25% +- 1.34% - auc: 61.28% +- 1.75%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 2.6612

SINKHORN RUN 3/10 - acc: 58.0% +- 1.15% - auc: 61.79% +- 1.19%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 2.9204

SINKHORN RUN 4/10 - acc: 58.2% +- 2.02% - auc: 63.03% +- 0.46%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 2.7422

SINKHORN RUN 5/10 - acc: 57.9% +- 1.73% - auc: 60.03% +- 2.0%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 2.6801

SINKHORN RUN 6/10 - acc: 59.5% +- 1.47% - auc: 62.0% +- 1.98%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 2.7053

SINKHORN RUN 7/10 - acc: 59.35% +- 2.44% - auc: 63.07% +- 2.87%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 2.8078

SINKHORN RUN 8/10 - acc: 59.05% +- 1.54% - auc: 63.29% 

## Run MIDA

In [11]:
mida_accs, mida_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputed = mida(data_missing_nans, num_layers=2, num_epochs=1500)
    imputed = np.where(missing_mask, data_missing, imputed)
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'\nMIDA RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%\n')
    mida_accs.append(acc.mean())
    mida_aucs.append(auc.mean())
print(f'\nMIDA GLOBAL - acc: {round(np.array(mida_accs).mean() * 100, 2)}% +- {round(np.array(mida_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(mida_aucs).mean() * 100, 2)}% +- {round(np.array(mida_aucs).std() * 100, 2)}%')

Stop training at epoch: 1500/1500, return best output

MIDA RUN 1/10 - acc: 59.0% +- 2.31% - auc: 62.36% +- 2.84%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 2/10 - acc: 61.35% +- 0.78% - auc: 64.6% +- 1.9%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 3/10 - acc: 60.1% +- 1.83% - auc: 64.03% +- 3.16%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 4/10 - acc: 61.15% +- 1.06% - auc: 65.54% +- 1.84%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 5/10 - acc: 60.15% +- 2.46% - auc: 63.32% +- 2.02%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 6/10 - acc: 60.15% +- 1.6% - auc: 63.17% +- 2.88%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 7/10 - acc: 59.65% +- 1.4% - auc: 62.93% +- 1.79%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 8/10 - acc: 59.2% +- 2.08% - auc: 62.58% +- 2.23%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 9/10 - acc: 59.

## Run T-tests

In [12]:
for model, metrics in {
        'MEDIAN': {'ACC': median_accs, 'AUC': median_aucs},
        'MEAN': {'ACC': mean_accs, 'AUC': mean_aucs},
        'KNN': {'ACC': knn_accs, 'AUC': knn_aucs},
        'GAIN': {'ACC': gain_accs, 'AUC': gain_aucs},
        'MICE': {'ACC': mice_accs, 'AUC': mice_aucs},
        'MISS': {'ACC': miss_accs, 'AUC': miss_aucs},
        'SOFT': {'ACC': soft_accs, 'AUC': soft_aucs},
        'SINK': {'ACC': sink_accs, 'AUC': sink_aucs},
        'MIDA': {'ACC': mida_accs, 'AUC': mida_aucs}}.items():
    for metric_name, metric in metrics.items():
        ours_metric = ours_accs if metric_name == 'ACC' else ours_aucs
        t, p = scipy.stats.ttest_ind(np.array(ours_metric), np.array(metric))
        if p <= .05:
            if t > 0:
                ## Our method is better
                print(f'Metric {metric_name} - OUR METHOD is significantly better than {model}')
            else:
                ## Theirs is better
                print(f'Metric {metric_name} - OUR METHOD is significantly worse than {model}')
        else:
            ## Else we are even
            print(f'Metric {metric_name} - OUR METHOD is even with {model}')

Metric ACC - OUR METHOD is significantly better than MEDIAN
Metric AUC - OUR METHOD is significantly better than MEDIAN
Metric ACC - OUR METHOD is significantly better than MEAN
Metric AUC - OUR METHOD is significantly better than MEAN
Metric ACC - OUR METHOD is significantly better than KNN
Metric AUC - OUR METHOD is significantly better than KNN
Metric ACC - OUR METHOD is significantly better than GAIN
Metric AUC - OUR METHOD is significantly better than GAIN
Metric ACC - OUR METHOD is significantly better than MICE
Metric AUC - OUR METHOD is significantly better than MICE
Metric ACC - OUR METHOD is significantly better than MISS
Metric AUC - OUR METHOD is significantly better than MISS
Metric ACC - OUR METHOD is significantly better than SOFT
Metric AUC - OUR METHOD is significantly better than SOFT
Metric ACC - OUR METHOD is significantly better than SINK
Metric AUC - OUR METHOD is significantly better than SINK
Metric ACC - OUR METHOD is significantly better than MIDA
Metric AUC -